## GSpace Simulation

GSpace uses a file named *GSpaceSettings.txt* specifying simulation parameters, see below the parameters used:

#### Simulation Settings
- **Data_filename**: `Simulated_sequences`
  Prefix for all output files.
- **Run_Number**: `1`
  Number of simulated datasets to generate.

---

#### Output File Format Settings
- **Output_Dir**: `../../TestExample_GSpace/results`
  Directory where output files will be saved.
- **Coordinate_file**: `true`
  Save a file with coordinates of sampled individuals.
- **Sequence_characteristics_file**: `true`
  Save additional sequence characteristics (e.g., mutations, coordinates).
- **Fasta**: `true`
  Export simulated sequences in FASTA format.
- **Fasta_Single_Line_Seq**: `true`
  Write each sequence on a single line in FASTA files.

---

#### Markers Settings
- **Ploidy**: `Haploid`
  Simulate haploid individuals.
- **Chromosome_number**: `1`
  Each individual has 1 chromosome.
- **Sequence_Size**: `1000`
  Each chromosome is 1000 nucleotides long.
- **Mutation_Model**: `HKY`
  Use the Hasegawa-Kishino-Yano (HKY) nucleotide substitution model.
- **Mutation_Rate**: `0.0005`
  Mutation rate per site per generation.

---

#### Recombination Settings
- **Recombination_Rate**: `0`
  No recombination within chromosomes.

---

#### Demographic Settings
- **Lattice_Size_X**: `20`
- **Lattice_Size_Y**: `20`
  Simulate a 20x20 grid (lattice) representing spatial structure.
- **Ind_Per_Pop**: `30`
  30 individuals per grid node (deme).

- **Dispersal_Distribution**: `uniform`
  Dispersal occurs uniformly to neighboring nodes.
- **Disp_Dist_Max**: `1,1`
  Maximum dispersal distance is 1 unit in both X and Y directions.
- **Total_Emigration_Rate**: `0.05`
  5% chance an individual migrates per generation.

---

#### Sample Settings
- **Ind_Per_Node_Sampled**: `5`
  Sample 5 individuals per selected node.
- **SampleCoordinateX**: `9,9,10,10`
- **SampleCoordinateY**: `12,13,12,13`
  Sampling occurs at 4 nodes: (9,12), (9,13), (10,12), (10,13).

> _Note_: The rectangular sampling settings are commented out.


In [1]:
import random

def generate_gspace_settings(output_dir=".",
                             lattice_size_x=20,
                             lattice_size_y=20,
                             num_sampled_nodes=4,
                             ind_per_node_sampled=5):
    """
    Generates a GSpaceSettings.txt file with random sampling coordinates.

    Parameters:
    - output_dir (str): Name of the output directory where the GSpaceSettings.txt file will be written.
    - lattice_size_x (int): Size of the lattice in X dimension.
    - lattice_size_y (int): Size of the lattice in Y dimension.
    - num_sampled_nodes (int): Number of distinct nodes to sample.
    - ind_per_node_sampled (int): Number of individuals per sampled node.
    """

    # Generate unique random coordinates
    sampled_positions = set()
    while len(sampled_positions) < num_sampled_nodes:
        x = random.randint(1, lattice_size_x)
        y = random.randint(1, lattice_size_y)
        sampled_positions.add((x, y))

    # Separate X and Y coordinates
    sample_x = ",".join(str(pos[0]) for pos in sampled_positions)
    sample_y = ",".join(str(pos[1]) for pos in sampled_positions)

    # GSpace settings content
    gspace_settings = f"""%%%%%%%% SIMULATION SETTINGS %%%%%%%%%%%%%%%
Data_filename=simulated_sequences
Run_Number=1

%%%%%%%% OUTPUT FILE FORMAT SETTINGS %%%%%%%
Output_Dir=../../TestExample_GSpace/results
%Coordinate_file=true
Sequence_characteristics_file=true
Fasta=true
Fasta_Single_Line_Seq=True

%%%%%%%% MARKERS SETTINGS %%%%%%%%%%%%%%%%%%
Ploidy=Haploid
Chromosome_number=1
Sequence_Size=1000
Mutation_Model=HKY
Mutation_Rate=0.0005

%%%%%%%% RECOMBINATION SETTINGS %%%%%%%%%%%%
Recombination_Rate=0

%%%%%%%% DEMOGRAPHIC SETTINGS %%%%%%%%%%%%%%
%% LATTICE
Lattice_Size_X={lattice_size_x}
Lattice_Size_Y={lattice_size_y}
Ind_Per_Pop=30

%% DISPERSAL
Dispersal_Distribution=uniform
Disp_Dist_Max=1,1
Total_Emigration_Rate=0.05

%%%%%%%% SAMPLE SETTINGS %%%%%%%%%%%%%%%%%%%
SampleCoordinateX={sample_x}
SampleCoordinateY={sample_y}
Ind_Per_Node_Sampled={ind_per_node_sampled}
"""

    # Write to file
    with open(f'{output_dir}/GSpaceSettings.txt', "w") as f:
        f.write(gspace_settings)

    print(f"GSpaceSettings.txt generated with random sampling coordinates in {output_dir}!")

## Generate GSpaceSettings.txt with random sampling positions

This script automates the creation of a `GSpaceSettings.txt` file for GSpace simulations, introducing random sampling coordinates within a defined lattice grid.

---
### 1. import required modules

```python
import random
```

import Python's built-in random integer generator `random`

---
### 2. Define parameters

```python
lattice_size_x = 20
lattice_size_y = 20
num_sampled_nodes = 4
ind_per_node_sampled = 5
```

- lattice_size_x / lattice_size_y: Define the grid size.
- num_sampled_nodes: Number of distinct grid nodes to sample.
- ind_per_node_sampled: Number of individuals to sample per node.
---
#### 3. Generate unique random coordinates

```python
sampled_positions = set()
while len(sampled_positions) < num_sampled_nodes:
    x = random.randint(1, lattice_size_x)
    y = random.randint(1, lattice_size_y)
    sampled_positions.add((x, y))
```

- Uses a set to ensure all sampled positions are unique.
- Randomly selects (x, y) coordinates within the lattice bounds until reaching the desired number of sampled nodes.
---
#### 4. Format coordinates for GSpace

```python
sample_x = ",".join(str(pos[0]) for pos in sampled_positions)
sample_y = ",".join(str(pos[1]) for pos in sampled_positions)
```

- Extracts X and Y coordinates separately
- Converts them into comma-separated strings matching GSpace’s expected input format:
  - SampleCoordinateX=...
  - SampleCoordinateY=...


## Generate BEAUti XML file for BEAST for sequences simulated with GSpace

### 1. The structure of a BEAUti XML file:

#### Header:
Specifying xml version and BEAST version:
```XML
<?xml version="1.0" standalone="yes"?>

<!-- Generated by BEAUTi v1.10.4 Prerelease #bc6cbd9                         -->
<!--       by Alexei J. Drummond, Andrew Rambaut and Marc A. Suchard         -->
<!--       Department of Computer Science, University of Auckland and        -->
<!--       Institute of Evolutionary Biology, University of Edinburgh        -->
<!--       David Geffen School of Medicine, University of California, Los Angeles-->
<!--       http://beast.community/                                           -->
<beast version="1.10.4">
...
</beast>
```
### Taxa:
List of Taxa to be analyzed (can also include dates/ages). includes locations.
```XML
<!-- ntax=65                                                                 -->
<taxa id"taxa">
    <taxon id="UNIQUE ID PER TAXON">
    <date value="..." direction="time direction (backwards if carbon dating)" units="years"/>
        <attr name="lat">
				X
			</attr>
			<attr name="long">
				Y
			</attr>
			<!-- START Multivariate diffusion model                                      -->
			<attr name="location">
				X Y
			</attr>

			<!-- END Multivariate diffusion model -->
 </taxon>
    ...
</taxa>
```

### Sequences:
The sequence alignment (each sequence refers to a taxon above).
```XML
<!-- ntax=65 nchar=10236                                                     -->
<alignment id="alignment" dataType="nucleotide">
    <sequence>
			<taxon idref="ID that references the sequence in Taxa"/>
			sequence ATCG one line
    </sequence>
    ...
</alignment>
```
### Patterns
patterns is the number of unique columns in the input multiple sequence alignment.
```XML
<!-- The unique patterns from 1 to end                                       -->
<!-- npatterns=                                                            -->
<patterns id="patterns" from="1" strip="false">
    <alignment idref="alignment"/>
</patterns>
```
### Population
```XML
<!-- A prior assumption that the population size has remained constant       -->
<!-- throughout the time spanned by the genealogy.                           -->
<constantSize id="constant" units="years">
    <populationSize>
        <parameter id="constant.popSize" value="1.0" lower="0.0"/>
    </populationSize>
</constantSize>
```
### Tree model

```XML
<!-- Generate a random starting tree under the coalescent process            -->
<coalescentSimulator id="startingTree">
    <coalescentSimulator>
        <taxa idref="taxonSet"/>
        <constantSize idref="constant"/>
    </coalescentSimulator>
    <taxa idref="taxa"/>
    <constantSize idref="constant"/>
</coalescentSimulator>


<!-- Generate a tree model                                                   -->
<treeModel id="treeModel">
    <coalescentTree idref="startingTree"/>
    <rootHeight>
        <parameter id="treeModel.rootHeight"/>
    </rootHeight>
    <nodeHeights internalNodes="true">
        <parameter id="treeModel.internalNodeHeights"/>
    </nodeHeights>
    <nodeHeights internalNodes="true" rootNode="true">
        <parameter id="treeModel.allInternalNodeHeights"/>
    </nodeHeights>
</treeModel>

<!-- Statistic for height of the root of the tree                            -->
<treeHeightStatistic id="rootHeight">
    <treeModel idref="treeModel"/>
</treeHeightStatistic>

<!-- Statistic for sum of the branch lengths of the tree (tree length)       -->
<treeLengthStatistic id="treeLength">
    <treeModel idref="treeModel"/>
</treeLengthStatistic>

<!-- Statistic for time of most recent common ancestor of tree               -->
<tmrcaStatistic id="age(root)" absolute="true">
    <treeModel idref="treeModel"/>
</tmrcaStatistic>

<!-- Taxon Sets                                                              -->

<tmrcaStatistic id="tmrca(taxonSet)" absolute="false" includeStem="true">
    <mrca>
        <taxa idref="taxonSet"/>
    </mrca>
    <treeModel idref="treeModel"/>
</tmrcaStatistic>
<tmrcaStatistic id="age(taxonSet)" absolute="true" includeStem="true">
    <mrca>
        <taxa idref="taxonSet"/>
    </mrca>
    <treeModel idref="treeModel"/>
</tmrcaStatistic>
<monophylyStatistic id="monophyly(taxonSet)">
    <mrca>
        <taxa idref="taxonSet"/>
    </mrca>
    <treeModel idref="treeModel"/>
</monophylyStatistic>

<!-- Generate a coalescent likelihood                                        -->
<coalescentLikelihood id="coalescent">
    <model>
        <constantSize idref="constant"/>
    </model>
    <populationTree>
        <treeModel idref="treeModel"/>
    </populationTree>
</coalescentLikelihood>
```
### Molecular Clock
#### Definition
The Uncorrelated Relaxed Clock (UCLN) model, introduced by Drummond, Ho, Phillips & Rambaut (2006) in PLoS Biology, is a molecular clock model used in Bayesian phylogenetics, implemented in BEAST.

---

In phylogenetics, a molecular clock estimates evolutionary time by assuming that genetic mutations accumulate at a certain rate. However, in real datasets:
- Evolutionary rates often vary among lineages due to differences in biology, environment, selection, etc.
- Strict clock (same rate across all branches) is often unrealistic.

The Uncorrelated Relaxed Clock addresses this by allowing:
- Each branch to have its own substitution rate.
- These rates are drawn independently (uncorrelated) from a specified statistical distribution (commonly:
    - LogNormal distribution → UCLD.
    - Or Exponential distribution → UCED.

---

#### Key Features of UCLN
1. Uncorrelated:
    - The rate on each branch is independent of neighboring branches.
    - No assumption that rates are inherited along the tree.
2. Relaxed:
    - Allows rate heterogeneity across branches.
    - Accommodates datasets where evolutionary rates vary due to differing life histories, environments, etc.
3. Statistical Distribution:
    - In UCLN, rates are drawn from a LogNormal distribution: $r_i \sim \text{LogNormal}(\mu, \sigma^2)$
    Where:
        - $r_i$ = rate on branch $i$ `<discretizedBranchRates id="branchRates">`
        - $\mu = mean$ (linked to overall clock rate) `<parameter id="ucld.mean" value="5.0E-4"/>`
        - $\sigma$ = standard deviation (degree of rate variation) `<parameter id="ucld.stdev" value="0.3333333333333333" lower="0.0"/>`
4.	Bayesian Framework:
    - BEAST jointly estimates:
    - The phylogenetic tree.
    - The rates on each branch.
    - Other evolutionary parameters.
    - Uncertainty is fully incorporated into posterior estimates.

---

#### When to Use UCLN?
- When you suspect rate variation among lineages.
- For datasets spanning different species, environments, or evolutionary pressures.
- To avoid biases from an overly restrictive strict clock.

---

#### Citation

Drummond AJ, Ho SYW, Phillips MJ, Rambaut A.
Relaxed Phylogenetics and Dating with Confidence.
[PLoS Biol 2006; 4(5): e88](https://doi.org/10.1371/journal.pbio.0040088).

```XML
<!-- The uncorrelated relaxed clock (Drummond, Ho, Phillips & Rambaut (2006) PLoS Biology 4, e88 )-->
<discretizedBranchRates id="branchRates">
    <treeModel idref="treeModel"/>
    <distribution>
        <logNormalDistributionModel meanInRealSpace="true">
            <mean>
                <parameter id="ucld.mean" value="5.0E-4"/>
            </mean>
            <stdev>
                <parameter id="ucld.stdev" value="0.3333333333333333" lower="0.0"/>
            </stdev>
        </logNormalDistributionModel>
    </distribution>
    <rateCategories>
        <parameter id="branchRates.categories"/>
    </rateCategories>
</discretizedBranchRates>

<rateStatistic id="meanRate" name="meanRate" mode="mean" internal="true" external="true">
    <treeModel idref="treeModel"/>
    <discretizedBranchRates idref="branchRates"/>
</rateStatistic>

<rateStatistic id="coefficientOfVariation" name="coefficientOfVariation" mode="coefficientOfVariation" internal="true" external="true">
    <treeModel idref="treeModel"/>
    <discretizedBranchRates idref="branchRates"/>
</rateStatistic>

<rateCovarianceStatistic id="covariance" name="covariance">
    <treeModel idref="treeModel"/>
    <discretizedBranchRates idref="branchRates"/>
</rateCovarianceStatistic>

```

### Substitution Model

```XML
<!-- The HKY substitution model (Hasegawa, Kishino & Yano, 1985)             -->
	<HKYModel id="hky">
		<frequencies>
			<frequencyModel dataType="nucleotide">
				<frequencies>
					<parameter id="frequencies" value="0.25 0.25 0.25 0.25"/>
				</frequencies>
			</frequencyModel>
		</frequencies>
		<kappa>
			<parameter id="kappa" value="2.0" lower="0.0"/>
		</kappa>
	</HKYModel>

	<!-- site model                                                              -->
	<siteModel id="siteModel">
		<substitutionModel>
			<HKYModel idref="hky"/>
		</substitutionModel>
	</siteModel>

	<!--                                                                         -->
	<statistic id="mu" name="mu">
		<siteModel idref="siteModel"/>
	</statistic>
```

### Multivariate Diffusion Model

1. Definition of the diffusion process

 ```XML
<multivariateDiffusionModel id="coordinates.diffusionModel">
		<precisionMatrix>
			<matrixParameter id="coordinates.precision">
				<parameter id="coordinates.precision.col1" value="0.05 0.002"/>
				<parameter id="coordinates.precision.col2" value="0.002 0.05"/>
			</matrixParameter>
		</precisionMatrix>
	</multivariateDiffusionModel>
```

This block defines the diffusion model for the spatial trait (coordinates).
- The precision matrix is the inverse of the variance-covariance matrix.
- The matrix:
    $$
    \begin{bmatrix}
    0.05 & 0.002 \\
    0.002 & 0.05
    \end{bmatrix}
    $$
controls:
- The rate of diffusion along each axis (latitude and longitude).
- The correlation between movements in both directions (here, a small positive correlation).

2. Prior on diffusion matrix
the block:
```XML
<multivariateWishartPrior id="coordinates.precisionPrior" df="2">
    <scaleMatrix>
        <matrixParameter>
            <parameter value="1.0 0.0"/>
            <parameter value="0.0 1.0"/>
        </matrixParameter>
    </scaleMatrix>
    <data>
        <parameter idref="coordinates.precision"/>
    </data>
</multivariateWishartPrior>
```
- defines a Wishart prior on the precision matrix (`df=2`refers to degrees of freedom).
- The scale matrix here is the identity matrix, representing a neutral prior expectation of equal diffusion in both dimensions without correlation.

3. Tree Likelihood for sequences data (unrelated to diffusion model, what is it doing here?)
This is unrelated to the spatial model but defines the likelihood of the tree given the sequence alignment and substitution model.
```XML
<!-- Likelihood for tree given sequence data                                 -->
<treeDataLikelihood id="treeLikelihood" useAmbiguities="false" usePreOrder="false">
    <partition>
        <patterns idref="patterns"/>
        <siteModel idref="siteModel"/>
    </partition>
    <treeModel idref="treeModel"/>
    <discretizedBranchRates idref="branchRates"/>
</treeDataLikelihood>
```
4. Likelihood of Trait Evolution (Coordinates)
This defines the likelihood of observing the spatial coordinates given the tree and the diffusion model.
```XML
	<traitDataLikelihood id="coordinates.traitLikelihood" traitName="coordinates" useTreeLength="true" scaleByTime="true" reportAsMultivariate="true" reciprocalRates="false" integrateInternalTraits="true">
		<multivariateDiffusionModel idref="coordinates.diffusionModel"/>
		<treeModel idref="treeModel"/>
		<traitParameter>
			<parameter id="leaf.coordinates"/>
		</traitParameter>
		<conjugateRootPrior>
			<meanParameter>
				<parameter value="0.0 0.0"/>
			</meanParameter>
			<priorSampleSize>
				<parameter value="0.000001"/>
			</priorSampleSize>
		</conjugateRootPrior>
	</traitDataLikelihood>
```
Key attributes:
- `scaleByTime="true"`: Diffusion scales with branch lengths (time-aware diffusion).
- `integrateInternalTraits="true"`: Internal node locations are integrated out rather than estimated explicitly.
- `conjugateRootPrior`: Specifies a weak prior on the root location at (0.0, 0.0).

5. Correlation and Variance-Covariance Extraction
Extracts:
- The correlation between latitude and longitude diffusion.
- The variance-covariance matrix (inverse of precision) for interpretation of diffusion rates.
```XML
<correlation id="coordinates.correlation" dimension1="1" dimension2="2">
		<matrixParameter idref="coordinates.precision"/>
	</correlation>
	<matrixInverse id="coordinates.varCovar">
		<matrixParameter idref="coordinates.precision"/>
	</matrixInverse>
```
6. Diffusion rate statistics
Calculates a summary diffusion rate statistic across the tree.
```XML
	<traitDataContinuousDiffusionStatistic id="coordinates.diffusionRate" traitName="coordinates" displacementScheme="linear" scalingScheme="dependent" weightingScheme="weighted">
		<traitDataLikelihood idref="coordinates.traitLikelihood"/>
	</traitDataContinuousDiffusionStatistic>
```
Parameters:
- `displacementScheme="linear"`: Linear displacement model.
- `scalingScheme="dependent"` and `weightingScheme="weighted"`: Control how diffusion is averaged over the tree.

### Operators
The root tag defines a list of operators.
```XML
<operators id="operators" optimizationSchedule="log">
    ...
</operators>
```
`optimizationSchedule="log"`: BEAST will log how well each operator performs and adjust their behavior during the run to optimize efficiency.

1. Substitution model operators
```XML
<scaleOperator scaleFactor="0.75" weight="1">
    <parameter idref="kappa"/>
</scaleOperator>
```
Proposes scaling moves on the kappa parameter (transition/transversion ratio in HKY model):

- `scaleFactor="0.75"`: Determines the size of proposed changes (multiplicative scaling).
- `weight="1"`: Frequency with which this operator is called relative to others.

```XML
<deltaExchange delta="0.01" weight="1">
    <parameter idref="frequencies"/>
</deltaExchange>
```
Adjusts nucleotide frequencies.
- `deltaExchange` proposes small changes ensuring the frequencies still sum to 1.
- `delta="0.01"`: Size of the proposed change.

2. Clock model operators
```XML
<scaleOperator scaleFactor="0.75" weight="3">
    <parameter idref="ucld.stdev"/>
</scaleOperator>
```
Adjusts the standard deviation of the Uncorrelated Lognormal Relaxed Clock (UCLD). Controls how much rate variation is allowed across branches.

```XML
<swapOperator size="1" weight="10" autoOptimize="false">
    <parameter idref="branchRates.categories"/>
</swapOperator>
<uniformIntegerOperator weight="10">
    <parameter idref="branchRates.categories"/>
</uniformIntegerOperator>
```
- These operators manage branch rate categories in discretized relaxed clock models.
- They propose swaps or changes in rate categories assigned to branches.

3. Tree topology operators
```XML
<subtreeLeap size="1.0" weight="30">
    <treeModel idref="treeModel"/>
</subtreeLeap>
```
- Proposes topological changes to the tree using a subtree leap move, which modifies parts of the tree.
- `weight="30"`: This is a frequently used operator because exploring tree space is critical.

```XML
<fixedHeightSubtreePruneRegraft weight="3">
    <treeModel idref="treeModel"/>
</fixedHeightSubtreePruneRegraft>
```
- Implements an SPR (Subtree Prune and Regraft) move.
- Keeps node heights fixed while changing the topology—helps efficiently explore tree space without altering divergence times.

4. Demographic parameter operators
```XML
<scaleOperator scaleFactor="0.75" weight="3">
    <parameter idref="constant.popSize"/>
</scaleOperator>
```
Adjusts the effective population size parameter in a constant coalescent demographic model.

5. Multivariate diffusion model operators
```XML
<precisionGibbsOperator weight="2">
    <wishartStatistics traitName="coordinates">
        <traitDataLikelihood idref="coordinates.traitLikelihood"/>
    </wishartStatistics>
    <multivariateWishartPrior idref="coordinates.precisionPrior"/>
</precisionGibbsOperator>
```
- Specialized operator for updating the precision matrix of the multivariate diffusion model (used for spatial trait evolution).
- Uses a Gibbs sampler, which is efficient for conjugate priors like the Wishart distribution.
- This operator proposes new precision matrices conditioned on current trait data.

### Generation of XML for BEAST given GSpace simulation (FASTA)

In [2]:
from Bio import SeqIO
import os

def generate_beast_xml(fasta_file, output_xml="output.xml", mutation_rate = 5.0E-4, chain_length=10000000):

    file_name = os.path.basename(fasta_file)

    records = list(SeqIO.parse(fasta_file, "fasta"))[1:]  # Skip ancestral sequence

    xml = '<?xml version="1.0" standalone="yes"?>\n'
    xml += '<beast version="1.10.4">'
    xml += " <!-- Generated by Rayane Ayoub AIT ALLAOUA - LIRMM 2025                  -->\n\n"

    # Taxa block
    xml += '\t<taxa id="taxa">\n'
    for record in records:
        seq_id = record.id
        parts = seq_id.split('_')
        idx = parts.index('coord')
        lat = float(parts[idx + 1]) + 0.1
        long = float(parts[idx + 2]) + 0.1
        xml += f'\t\t<taxon id="{seq_id}">\n'
        xml += f'\t\t\t<attr name="lat">{lat}</attr>\n'
        xml += f'\t\t\t<attr name="long">{long}</attr>\n'
        xml += f"""			<!-- START Multivariate diffusion model                                      -->
			<attr name="coordinates">
				{lat} {long}
			</attr>

			<!-- END Multivariate diffusion model                                        -->"""
        xml += f'\t\t</taxon>\n'
    xml += '\t</taxa>\n\n'

    # taxonSet block
    xml += '\t<taxa id="taxonSet">\n'
    for record in records:
        seq_id = record.id
        xml += f'\t\t<taxon idref="{seq_id}"/>\n'
    xml += '</taxa>\n\n'

    # Alignment block
    xml += f"\t<!-- ntax={len(records)} | nchar={len(records[0])}-->\n"
    xml += '\t<alignment id="alignment" dataType="nucleotide">\n'
    for record in records:
        xml += f'\t\t<sequence>\n'
        xml += f'\t\t\t<taxon idref="{record.id}"/>\n'
        xml += f'\t\t\t{str(record.seq)}\n'
        xml += f'\t\t</sequence>\n'
    xml += '\t</alignment>\n\n'

    # patterns block
    xml += """  <!-- The unique patterns from 1 to end                                       -->
        <patterns id="patterns" from="1" strip="false">
            <alignment idref="alignment"/>
	    </patterns>\n\n"""

    # Population size block
    xml += """  <!-- A prior assumption that the population size has remained constant       -->
	    <!-- throughout the time spanned by the genealogy.                           -->
        <constantSize id="constant" units="years">
	        <populationSize>
                <parameter id="constant.popSize" value="1.0" lower="0.0"/>
	        </populationSize>
	    </constantSize>\n\n"""

    # Starting tree block
    xml += """  <!-- Generate a random starting tree under the coalescent process            -->
	    <coalescentSimulator id="startingTree">
	        <coalescentSimulator>
	            <taxa idref="taxonSet"/>
	            <constantSize idref="constant"/>
	        </coalescentSimulator>
	        <taxa idref="taxa"/>
	        <constantSize idref="constant"/>
	    </coalescentSimulator>\n\n"""

    # Tree model block
    xml += """  <!-- Generate a tree model                                                   -->
	    <treeModel id="treeModel">
	        <coalescentTree idref="startingTree"/>
	        <rootHeight>
	            <parameter id="treeModel.rootHeight"/>
	        </rootHeight>
	        <nodeHeights internalNodes="true">
	            <parameter id="treeModel.internalNodeHeights"/>
	        </nodeHeights>
	        <nodeHeights internalNodes="true" rootNode="true">
	            <parameter id="treeModel.allInternalNodeHeights"/>
	        </nodeHeights>
	    </treeModel>

	<!-- Statistic for height of the root of the tree                            -->
	<treeHeightStatistic id="rootHeight">
		<treeModel idref="treeModel"/>
	</treeHeightStatistic>

	<!-- Statistic for sum of the branch lengths of the tree (tree length)       -->
	<treeLengthStatistic id="treeLength">
		<treeModel idref="treeModel"/>
	</treeLengthStatistic>

	<!-- Statistic for time of most recent common ancestor of tree               -->
	<tmrcaStatistic id="age(root)" absolute="true">
		<treeModel idref="treeModel"/>
	</tmrcaStatistic>

	<!-- Taxon Sets                                                              -->

	<tmrcaStatistic id="tmrca(taxonSet)" absolute="false" includeStem="true">
		<mrca>
			<taxa idref="taxonSet"/>
		</mrca>
		<treeModel idref="treeModel"/>
	</tmrcaStatistic>
	<tmrcaStatistic id="age(taxonSet)" absolute="true" includeStem="true">
		<mrca>
			<taxa idref="taxonSet"/>
		</mrca>
		<treeModel idref="treeModel"/>
	</tmrcaStatistic>
	<monophylyStatistic id="monophyly(taxonSet)">
		<mrca>
			<taxa idref="taxonSet"/>
		</mrca>
		<treeModel idref="treeModel"/>
	</monophylyStatistic>\n\n
    """

    # Coalescent likelihood block
    xml += """
    	<!-- Generate a coalescent likelihood                                        -->
	<coalescentLikelihood id="coalescent">
		<model>
			<constantSize idref="constant"/>
		</model>
		<populationTree>
			<treeModel idref="treeModel"/>
		</populationTree>
	</coalescentLikelihood>\n\n
    """

    # Molecular clock
    xml += f"""
    <!-- The uncorrelated relaxed clock (Drummond, Ho, Phillips & Rambaut (2006) PLoS Biology 4, e88 )-->
	<discretizedBranchRates id="branchRates">
		<treeModel idref="treeModel"/>
		<distribution>
			<logNormalDistributionModel meanInRealSpace="true">
				<mean>
					<parameter id="ucld.mean" value="{mutation_rate}"/>
				</mean>
				<stdev>
					<parameter id="ucld.stdev" value="0.3333333333333333" lower="0.0"/>
				</stdev>
			</logNormalDistributionModel>
		</distribution>
		<rateCategories>
			<parameter id="branchRates.categories"/>
		</rateCategories>
	</discretizedBranchRates>

	<rateStatistic id="meanRate" name="meanRate" mode="mean" internal="true" external="true">
		<treeModel idref="treeModel"/>
		<discretizedBranchRates idref="branchRates"/>
	</rateStatistic>

	<rateStatistic id="coefficientOfVariation" name="coefficientOfVariation" mode="coefficientOfVariation" internal="true" external="true">
		<treeModel idref="treeModel"/>
		<discretizedBranchRates idref="branchRates"/>
	</rateStatistic>

	<rateCovarianceStatistic id="covariance" name="covariance">
		<treeModel idref="treeModel"/>
		<discretizedBranchRates idref="branchRates"/>
	</rateCovarianceStatistic>\n\n"""

    # Substitution Model (HKY)
    xml += "<!-- The HKY substitution model (Hasegawa, Kishino & Yano, 1985)             -->\n"
    xml += '\t<HKYModel id="hky">\n'
    xml += '\t\t<kappa>\n\t\t\t<parameter id="kappa" value="2.0" lower="0.0"/>\n\t\t</kappa>\n'
    xml += """
		<frequencies>
			<frequencyModel dataType="nucleotide">
				<frequencies>
					<parameter id="frequencies" value="0.25 0.25 0.25 0.25"/>
				</frequencies>
			</frequencyModel>
		</frequencies>
    """
    xml += '\t</HKYModel>\n\n'

    # Site Model
    xml += '\t<siteModel id="siteModel">\n'
    xml += '\t\t<substitutionModel>\n'
    xml += '\t\t\t<HKYModel idref="hky"/>'
    xml += '\t\t</substitutionModel>\n'
    xml += '\t</siteModel>\n\n'
    xml += """
    	<statistic id="mu" name="mu">
		    <siteModel idref="siteModel"/>
	    </statistic>\n\n"""

    # Multivariate diffusion model
    xml += """
    <!-- START Multivariate diffusion model                                      -->

	<multivariateDiffusionModel id="coordinates.diffusionModel">
		<precisionMatrix>
			<matrixParameter id="coordinates.precision">
				<parameter id="coordinates.precision.col1" value="0.05 0.002"/>
				<parameter id="coordinates.precision.col2" value="0.002 0.05"/>
			</matrixParameter>
		</precisionMatrix>
	</multivariateDiffusionModel>

	<multivariateWishartPrior id="coordinates.precisionPrior" df="2">
		<scaleMatrix>
			<matrixParameter>
				<parameter value="1.0 0.0"/>
				<parameter value="0.0 1.0"/>
			</matrixParameter>
		</scaleMatrix>
		<data>
			<parameter idref="coordinates.precision"/>
		</data>
	</multivariateWishartPrior>\n
    """

    # trait data likelihood
    xml += """
    <traitDataLikelihood id="coordinates.traitLikelihood" traitName="coordinates" useTreeLength="true" scaleByTime="true" reportAsMultivariate="true" reciprocalRates="false" integrateInternalTraits="true">
		<multivariateDiffusionModel idref="coordinates.diffusionModel"/>
		<treeModel idref="treeModel"/>
		<traitParameter>
			<parameter id="leaf.coordinates"/>
		</traitParameter>
		<conjugateRootPrior>
			<meanParameter>
				<parameter value="0.0 0.0"/>
			</meanParameter>
			<priorSampleSize>
				<parameter value="0.000001"/>
			</priorSampleSize>
		</conjugateRootPrior>
	</traitDataLikelihood>
	<correlation id="coordinates.correlation" dimension1="1" dimension2="2">
		<matrixParameter idref="coordinates.precision"/>
	</correlation>
	<matrixInverse id="coordinates.varCovar">
		<matrixParameter idref="coordinates.precision"/>
	</matrixInverse>
	<traitDataContinuousDiffusionStatistic id="coordinates.diffusionRate" traitName="coordinates" displacementScheme="linear" scalingScheme="dependent" weightingScheme="weighted">
		<traitDataLikelihood idref="coordinates.traitLikelihood"/>
	</traitDataContinuousDiffusionStatistic>

	<!-- END Multivariate diffusion model                                        -->\n\n"""

    # Tree likelihood block

    xml += """
    	<treeDataLikelihood id="treeLikelihood" useAmbiguities="false" usePreOrder="false">
		<partition>
			<patterns idref="patterns"/>
			<siteModel idref="siteModel"/>
		</partition>
		<treeModel idref="treeModel"/>
		<discretizedBranchRates idref="branchRates"/>
	</treeDataLikelihood>\n\n"""

    # Operators block

    xml += """
    	<!-- Define operators                                                        -->
	<operators id="operators" optimizationSchedule="log">
		<scaleOperator scaleFactor="0.75" weight="1">
			<parameter idref="kappa"/>
		</scaleOperator>
		<deltaExchange delta="0.01" weight="1">
			<parameter idref="frequencies"/>
		</deltaExchange>
		<scaleOperator scaleFactor="0.75" weight="3">
			<parameter idref="ucld.stdev"/>
		</scaleOperator>
		<swapOperator size="1" weight="10" autoOptimize="false">
			<parameter idref="branchRates.categories"/>
		</swapOperator>
		<uniformIntegerOperator weight="10">
			<parameter idref="branchRates.categories"/>
		</uniformIntegerOperator>
		<subtreeLeap size="1.0" weight="30">
			<treeModel idref="treeModel"/>
		</subtreeLeap>
		<fixedHeightSubtreePruneRegraft weight="3">
			<treeModel idref="treeModel"/>
		</fixedHeightSubtreePruneRegraft>
		<scaleOperator scaleFactor="0.75" weight="3">
			<parameter idref="constant.popSize"/>
		</scaleOperator>

		<!-- START Multivariate diffusion model                                      -->
		<precisionGibbsOperator weight="2">
			<wishartStatistics traitName="coordinates">
				<traitDataLikelihood idref="coordinates.traitLikelihood"/>
			</wishartStatistics>
			<multivariateWishartPrior idref="coordinates.precisionPrior"/>
		</precisionGibbsOperator>

		<!-- END Multivariate diffusion model                                        -->

	</operators> \n\n"""
    # MCMC Block
    xml += f'\t	<mcmc id="mcmc" chainLength="{chain_length}" autoOptimize="true" operatorAnalysis="{file_name}.ops">\n'
    xml += f"""
    		<joint id="joint">
			<prior id="prior">
				<logNormalPrior mu="1.0" sigma="1.25" offset="0.0">
					<parameter idref="kappa"/>
				</logNormalPrior>
				<dirichletPrior alpha="1.0" sumsTo="1.0">
					<parameter idref="frequencies"/>
				</dirichletPrior>
				<exponentialPrior mean="0.3333333333333333" offset="0.0">
					<parameter idref="ucld.stdev"/>
				</exponentialPrior>
				<gammaPrior shape="0.001" scale="1000.0" offset="0.0">
					<parameter idref="constant.popSize"/>
				</gammaPrior>
				<coalescentLikelihood idref="coalescent"/>


				<discretizedBranchRates idref="branchRates"/>

				<!-- START Multivariate diffusion model                                      -->
				<multivariateWishartPrior idref="coordinates.precisionPrior"/>

				<!-- END Multivariate diffusion model                                        -->

			</prior>
			<likelihood id="likelihood">
				<treeDataLikelihood idref="treeLikelihood"/>

				<!-- START Multivariate diffusion model                                      -->
				<traitDataLikelihood idref="coordinates.traitLikelihood"/>

				<!-- END Multivariate diffusion model                                        -->

			</likelihood>
		</joint>
		<operators idref="operators"/>

		<!-- write log to screen                                                     -->
		<log id="screenLog" logEvery="1000">
			<column label="Joint" dp="4" width="12">
				<joint idref="joint"/>
			</column>
			<column label="Prior" dp="4" width="12">
				<prior idref="prior"/>
			</column>
			<column label="Likelihood" dp="4" width="12">
				<likelihood idref="likelihood"/>
			</column>
			<column label="age(root)" sf="6" width="12">
				<tmrcaStatistic idref="age(root)"/>
			</column>
		</log>

		<!-- write log to file                                                       -->
		<log id="fileLog" logEvery="1000" fileName="{file_name}.log" overwrite="false">
			<joint idref="joint"/>
			<prior idref="prior"/>
			<likelihood idref="likelihood"/>
			<treeHeightStatistic idref="rootHeight"/>
			<tmrcaStatistic idref="age(root)"/>
			<treeLengthStatistic idref="treeLength"/>
			<tmrcaStatistic idref="tmrca(taxonSet)"/>
			<tmrcaStatistic idref="age(taxonSet)"/>
			<parameter idref="constant.popSize"/>
			<parameter idref="kappa"/>
			<parameter idref="frequencies"/>
			<parameter idref="ucld.mean"/>
			<parameter idref="ucld.stdev"/>
			<rateStatistic idref="meanRate"/>
			<rateStatistic idref="coefficientOfVariation"/>
			<rateCovarianceStatistic idref="covariance"/>

			<!-- START Multivariate diffusion model                                      -->
			<matrixParameter idref="coordinates.precision"/>
			<correlation idref="coordinates.correlation"/>
			<matrixInverse idref="coordinates.varCovar"/>
			<traitDataContinuousDiffusionStatistic idref="coordinates.diffusionRate"/>

			<!-- END Multivariate diffusion model                                        -->

			<treeDataLikelihood idref="treeLikelihood"/>
			<discretizedBranchRates idref="branchRates"/>

			<!-- START Multivariate diffusion model                                      -->
			<traitDataLikelihood idref="coordinates.traitLikelihood"/>

			<!-- END Multivariate diffusion model                                        -->

			<coalescentLikelihood idref="coalescent"/>

		</log>

		<!-- write tree log to file                                                  -->
		<logTree id="treeFileLog" logEvery="1000" nexusFormat="true" fileName="{file_name}.trees" sortTranslationTable="true">
			<treeModel idref="treeModel"/>
			<trait name="rate" tag="rate">
				<discretizedBranchRates idref="branchRates"/>
			</trait>
			<joint idref="joint"/>

			<!-- START Ancestral state reconstruction                                    -->
			<trait name="coordinates" tag="coordinates">
				<traitDataLikelihood idref="coordinates.traitLikelihood"/>
			</trait>

			<!-- END Ancestral state reconstruction                                      -->


			<!-- START Multivariate diffusion model                                      -->
			<multivariateDiffusionModel idref="coordinates.diffusionModel"/>
			<traitDataLikelihood idref="coordinates.traitLikelihood"/>

			<!-- END Multivariate diffusion model                                        -->

		</logTree>

		<!-- write state of Markov chain to checkpoint file                          -->
		<logCheckpoint id="checkpointFileLog" checkpointEvery="{chain_length/10}" checkpointFinal="{chain_length}" fileName="{file_name}.chkpt" overwrite="false"/>
	</mcmc>"""

    xml += '</beast>\n'

    with open(output_xml, 'w') as f:
        f.write(xml)

    print(f"BEAST XML generated: {output_xml}")

### Workflow

#### 1. Set the working directory

Set the working where your GSpaceSettings.txt file will be generated. further results of the analysis will be found here as well.

In [3]:
import os

# Check if we are already in 'Tests' directory
if os.path.basename(os.getcwd()) == "Tests":
    print("Already in the 'Tests' directory!")
    print(f"Current working directory: {os.getcwd()}")
else:
    # Check if 'Tests' exists in current directory
    if not os.path.exists("Tests"):
        os.makedirs("Tests")
    os.chdir("Tests")
    print(f"Moved into 'Tests' directory. Current working directory: {os.getcwd()}")

Moved into 'Tests' directory. Current working directory: /Users/ayoubrayaneaitallaoua/Documents/LIRMM/Pathogen_Dispersal_Rate_Estimation/Tests


#### 2. Generate GSpace Settings File
Automate the creation of `GSpaceSettings.txt` with random sampling coordinates.

In [4]:
import subprocess

def run_gspace(gspace_dir="../GSpace/build/GSpace",
               lattice_size_x=20,
               lattice_size_y=20,
               num_sampled_nodes=5,
               ind_per_node_sampled=5):

    # 1. Generate GSpaceSettings.txt
    generate_gspace_settings(
        output_dir=".",
        lattice_size_x=lattice_size_x,
        lattice_size_y=lattice_size_y,
        num_sampled_nodes=num_sampled_nodes,
        ind_per_node_sampled=ind_per_node_sampled
    )

    # 2. Get GSpace executable
    gspace_executable = f"{gspace_dir}"

    # 3. Check if executable exists and run
    if os.path.isfile(gspace_executable):
        subprocess.run([gspace_executable])
    else:
        print(f"Error: {gspace_executable} not found.")

#### 3. Run GSpace simulation

In [5]:
run_gspace(gspace_dir="../../GSpace/build/GSpace")

GSpaceSettings.txt generated with random sampling coordinates in .!
reading settings file : GSpaceSettings.txt

Random assignation 1 chromosome MRCA nucleotidic states. Press any key to resume.


         This is  GSpace  v0.1 (Built on Apr 22 2025 at 15:19:09)    
               (Virgoulay et al. 2020 Bioinformatics)                       
         an exact coalescent simulator of genetic /  genomic data           
            under generalized models of isolation by distance               
Settings summary : Generic output filename is simulated_sequences
 Simulation of 1 data sets
   with 1 chromosomes / independant loci with 1000 linked sites /  loci each. 
 Mutation model is hky
   with a mutation rate of 0.0005 mutations per site per generation.
   and a recombination rate of 0 between adjacent sites per generation.
Homogeneous sample of size (1x1)*5 = 5 haploid individuals 
evolving on a 20 x 20 lattice with reflecting boundaries
  where each node carries 30 individuals.
Dispersa

### 4. Generate BEAST XML

1. search for fasta files (*.fa): if multiple `.fa` files are returned, specify the index of desired file to use when calling `generate_beast_xml(fasta_file=fa_files[index])`.
2. generate the XML based on the chosen fasta file

In [6]:
# 1. find the .fa file
import glob
import os

fa_files = [os.path.abspath(f) for f in glob.glob("*.fa")]
print(fa_files)

['/Users/ayoubrayaneaitallaoua/Documents/LIRMM/Pathogen_Dispersal_Rate_Estimation/Tests/simulated_sequences_Fasta_1.fa']


In [7]:
generate_beast_xml(fasta_file=fa_files[0])

BEAST XML generated: output.xml


#### 5. Run BEAST using the generated XML

In [8]:
import subprocess
import os

def run_beast(xml_file, beast_path):

    if not os.path.isfile(xml_file):
        raise FileNotFoundError(f"XML file not found: {xml_file}")

    if not os.path.isfile(beast_path):
        raise FileNotFoundError(f"BEAST executable not found: {beast_path}")

    try:
        result = subprocess.run(
            [beast_path, xml_file],
            check=True
        )
        print("BEAST ran successfully.")
        return result.returncode
    except subprocess.CalledProcessError as e:
        print(f"BEAST failed with return code {e.returncode}")
        return e.returncode

In [ ]:
print(os.getcwd())
run_beast(xml_file="./output.xml",beast_path="../../BEAST/bin/beast")

/Users/ayoubrayaneaitallaoua/Documents/LIRMM/Pathogen_Dispersal_Rate_Estimation/Tests

  BEAST X v10.5.0-beta5 Prerelease #1d511b10c2, 2002-2024
       Bayesian Evolutionary Analysis Sampling Trees
                 Designed and developed by
   Alexei J. Drummond, Andrew Rambaut and Marc A. Suchard
                              
               Department of Computer Science
                   University of Auckland
                  alexei@cs.auckland.ac.nz
                              
             Institute of Ecology and Evolution
                  University of Edinburgh
                     a.rambaut@ed.ac.uk
                              
              David Geffen School of Medicine
           University of California, Los Angeles
                     msuchard@ucla.edu
                              
                Downloads, Help & Resources:
                  	http://beast.community
                              
Source code distributed under the GNU Lesser General Public Lice

Base CDI is dr.evomodel.treedatalikelihood.continuous.cdi.ContinuousDiffusionIntegrator.Basic



Citations for this analysis: 

FRAMEWORK
BEAST primary citation:
	Suchard MA, Lemey P, Baele G, Ayres DL, Drummond AJ, Rambaut A (2018) Bayesian phylogenetic and phylodynamic data integration using BEAST 1.10. Virus Evolution. vey016. DOI:10.1093/ve/vey016
Multivariate diffusion model (first citation) with efficiently integrated internal traits (second citation):
	Lemey P, Rambaut A, Welch JJ, Suchard MA (2010) Phylogeography takes a relaxed random walk in continous space and time. Molecular Biology and Evolution. 27, 1877-1885
	Pybus OG, Suchard MA, Lemey P, Bernadin F, Rambaut A, Crawford FW, Gray RR, Arinaminpathy N, Stramer S, Busch MP, Delwart E (2012) Unifying the spatial epidemiology and evolution of emerging epidemics. Proceedings of the National Academy of Sciences. 109, 15066-15071

HIGH-PERFORMANCE COMPUTING
BEAGLE citation:
	Ayres DL, Cummings MP, Baele G, Darling AE, Lewis PO, Swofford DL, Huelsenbeck JP, Lemey P, Rambaut A, Suchard MA (2019) BEAGLE 3: Improved performanc